# Spark R 패키지 로딩

- SparkR 패키지가 설치된 docker 이미지이기 때문에 설치 과정이 필요없음.

In [1]:
library(SparkR)


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window

The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union



In [3]:
sparkR.session()

Spark package found in SPARK_HOME: /usr/local/spark


Launching java with spark-submit command /usr/local/spark/bin/spark-submit   sparkr-shell /tmp/RtmpDC8Qzm/backend_portd02d894592 


Java ref type org.apache.spark.sql.SparkSession id 1 

# SparkDataFrames으로 데이터를 읽어오기 

In [4]:
df <- as.DataFrame(faithful)

# Displays the first part of the SparkDataFrame
head(df)

eruptions,waiting
3.600,79
1.800,54
3.333,74
2.283,62
4.533,85
2.883,55


# Selecting rows, columns 과 Grouping, Aggregation 

In [5]:
head(select(df, df$eruptions))

eruptions
3.600
1.800
3.333
2.283
4.533
2.883


In [6]:
head(select(df, "eruptions"))

eruptions
3.600
1.800
3.333
2.283
4.533
2.883


In [7]:
head(filter(df, df$waiting < 50))

eruptions,waiting
1.750,47
1.750,47
1.867,48
1.750,48
2.167,48
2.100,49


In [8]:
waiting_counts <- summarize(groupBy(df, df$waiting), count = n(df$waiting))
head(arrange(waiting_counts, desc(waiting_counts$count)))

waiting,count
78,15
83,14
81,13
77,12
82,12
79,10


# Large dataset using dapply or dapplyCollect

In [9]:
schema <- structType(structField("eruptions", "double"), structField("waiting", "double"),
                     structField("waiting_secs", "double"))
df1 <- dapply(df, function(x) { x <- cbind(x, x$waiting * 60) }, schema)
head(collect(df1))

eruptions,waiting,waiting_secs
3.600,79,4740
1.800,54,3240
3.333,74,4440
2.283,62,3720
4.533,85,5100
2.883,55,3300


In [10]:
# Note that we can apply UDF to DataFrame and return a R's data.frame
ldf <- dapplyCollect(
         df,
         function(x) {
           x <- cbind(x, "waiting_secs" = x$waiting * 60)
         })
head(ldf, 3)

eruptions,waiting,waiting_secs
3.600,79,4740
1.800,54,3240
3.333,74,4440


# Generalized linear regression

In [14]:
training <- read.df("sample_multiclass_classification_data.txt", source = "libsvm")

# Fit a generalized linear model of family "gaussian" with spark.glm
df_list <- randomSplit(training, c(7, 3), 2)
gaussianDF <- df_list[[1]]
gaussianTestDF <- df_list[[2]]
gaussianGLM <- spark.glm(gaussianDF, label ~ features, family = "gaussian")

# Model summary
summary(gaussianGLM)


Deviance Residuals: 
(Note: These are approximate quantiles with relative error <= 0.01)
     Min        1Q    Median        3Q       Max  
-1.65951  -0.48982  -0.11214   0.56093   1.47146  

Coefficients:
              Estimate  Std. Error   t value    Pr(>|t|)
(Intercept)   0.886604    0.080965  10.95045  0.0000e+00
features_0   -0.060493    0.406325  -0.14888  8.8195e-01
features_1   -0.613922    0.281482  -2.18103  3.1502e-02
features_2    1.677997    0.701195   2.39305  1.8556e-02
features_3   -2.035859    0.475488  -4.28162  4.2331e-05

(Dispersion parameter for gaussian family taken to be 0.4809826)

    Null deviance: 70.538  on 105  degrees of freedom
Residual deviance: 48.579  on 101  degrees of freedom
AIC: 230.1

Number of Fisher Scoring iterations: 1
